### ```Model```

### ```Model II```

In [ ]:
model_path = "/tmp/model"

tf.saved_model.save(model, model_path)

In [ ]:
filemega = lambda filepath : os.path.getsize(filepath) / float(2**20)

optm = [tf.lite.Optimize.DEFAULT]

tflite_model = Tensolite(model, optm=optm, filename="normal_model")

round(filemega("/content/normal_model.tflite"), 3)

In [ ]:
def mini_datagen(main_data, batch=1, take=100):
  for t in dataset.from_tensor_slices(main_data).batch(batch).take(take):
    yield [t]

In [ ]:
tflite_model = lite.from_saved_model(model_path)

tflite_model.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model.representative_dataset = mini_datagen

In [ ]:
tflite_model.target_spec.supported_ops

In [ ]:
tflite_model.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

tflite_model.target_spec.supported_ops

In [ ]:
tflite_model.inference_input_type, tflite_model.inference_output_type

In [ ]:
tflite_model.inference_input_type = tf.uint8

tflite_model.inference_output_type = tf.uint8

tflite_model.inference_input_type, tflite_model.inference_output_type